In [ ]:
import pandas as pd      
import os
import time
import glob
from pymongo import MongoClient
from pprint import pprint
from dotenv import load_dotenv
from datetime import datetime,timezone,timedelta
from pathlib import Path

In [ ]:
# Import our utils libraries
from utils.mongo import (
    connect_to_db,
    get_coinpairs,
    get_binance_timeframes_list,
    get_pair_data,
    #get_coinpair_df_mongo,
    #get_google_trends_mongo,
    #get_fng_mongo,
    get_keywords,
    get_main_coin,
    update_trends_db,
    #get_last_coinpair_update
)
from utils.binance import (
    get_coinpair_kline
)
from utils.google_trends import (
    process_trends,
    get_google_trends
)

In [ ]:
# Load Environment Variables
load_dotenv()

# Connect to the DB
db = connect_to_db(os.getenv('MDB_CONNECTION_STRING'))

5.0.6


In [ ]:
# Gets Binance Api Key and Api Secret
BINANCE_API_KEY = os.getenv('BINANCE_API_KEY')
BINANCE_API_SECRET = os.getenv('BINANCE_API_SECRET')

In [ ]:
async def process_binance_data(db, coinpairs):
    # Get timeframes from database
    binance_timeframes_list = get_binance_timeframes_list(db)
    
    # Get pair info to update DB
    complete_pair_tf = get_pair_data(db, binance_timeframes_list, coinpairs)

    headers=["open_time", "open", "high", "low","close","volume","close_time","quote_asset_volume","numer_trades","taker_base_volume","taker_quote_volume","ignore"]
    for query_pair in complete_pair_tf:
        collection_name = query_pair[1]+"_"+query_pair[0]
        # Get the data from Binance
        kline_list = await get_coinpair_kline(BINANCE_API_KEY, BINANCE_API_SECRET, query_pair[1], query_pair[0], query_pair[2])
        # If it is updating the db then delete the first item as it repeats
        if query_pair[4] == "is_update":
            kline_list.pop(0)
            print(f"Updating {len(kline_list)} items to {collection_name}..")
        else:
            print(f"Adding {len(kline_list)} items to {collection_name}..")
        # If there is items to update process
        if len(kline_list) > 0:
            kline_df = pd.DataFrame(kline_list, columns=headers)
            kline_df['open_time'] = kline_df['open_time'].values.astype(dtype='datetime64[ms]')
            kline_df['close_time'] = kline_df['close_time'].values.astype(dtype='datetime64[ms]')
            kline_df[["open", "high", "low","close","volume","quote_asset_volume","taker_base_volume","taker_quote_volume"]] = kline_df[["open", "high", "low","close","volume","quote_asset_volume","taker_base_volume","taker_quote_volume"]].astype(float)
            kline_df = kline_df.drop(columns=['ignore'])
            kline_dict = kline_df.to_dict("records")
            # Save to DB
            db[query_pair[3]].insert_many(kline_dict)
    print(f"Done :)")


In [ ]:
# Function to initialize the databse with the coinpairs registered
def update_google_trends(db, coinpairs):
    for coinpair in coinpairs:
        coin_name = get_main_coin(db, coinpair)
        print(f"Updating trends for {coin_name}..")
        keywords = get_keywords(db, coinpair)
        get_google_trends(db, coinpair, keywords, coin_name)
            
    print(f"Done pulling trends.")

In [ ]:
# Get coinpair list from database
coinpairs = get_coinpairs(db)

# Update DB with the latest binance Data
await process_binance_data(db, coinpairs)

# Update Google Trends data
update_google_trends(db, coinpairs)

Updating 2 items to ADABUSD_1m..
Updating 2 items to SOLBUSD_1m..
Updating 2 items to BTCBUSD_1m..
Updating 2 items to ETHBUSD_1m..
Updating 2 items to LUNABUSD_1m..
Updating 2 items to DOGEBUSD_1m..
Updating 2 items to MATICBUSD_1m..
Updating 2 items to MANABUSD_1m..
Updating 0 items to ADABUSD_5m..
Updating 0 items to SOLBUSD_5m..
Updating 0 items to BTCBUSD_5m..
Updating 0 items to ETHBUSD_5m..
Updating 0 items to LUNABUSD_5m..
Updating 0 items to DOGEBUSD_5m..
Updating 0 items to MATICBUSD_5m..
Updating 0 items to MANABUSD_5m..
Updating 0 items to ADABUSD_30m..
Updating 0 items to SOLBUSD_30m..
Updating 0 items to BTCBUSD_30m..
Updating 0 items to ETHBUSD_30m..
Updating 0 items to LUNABUSD_30m..
Updating 0 items to DOGEBUSD_30m..
Updating 0 items to MATICBUSD_30m..
Updating 0 items to MANABUSD_30m..
Updating 0 items to ADABUSD_1h..
Updating 0 items to SOLBUSD_1h..
Updating 0 items to BTCBUSD_1h..
Updating 0 items to ETHBUSD_1h..
Updating 0 items to LUNABUSD_1h..
Updating 0 items to